In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, datetime, timedelta
#Con esto podemos echar un vistazo a código HTML directamente en la Notebook
from IPython.display import HTML, display
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_predict, cross_val_score
from sklearn.feature_selection import rfe, SelectKBest, f_classif, f_regression
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
%matplotlib inline

In [33]:
file = 'appointments2016.csv'
data = pd.read_csv(file)
data.shape

(110527, 117)

In [34]:
data.shape

(110527, 117)

## BASELINE 2016

In [35]:
1 - data['No-show'].sum()/len(data['No-show'])

0.798067440534892

In [36]:
features_to_ignore = ['DateAppointment','TimeScheduled', 'DateScheduled', 'AppointmentID',\
                      'PatientId', 'Neighbourhood', 'day_of_week', 'event', 'No-show']
valid_features = [c for c in data.columns if c not in features_to_ignore]
len(valid_features)

108

In [37]:

X = data[valid_features]
y = data['No-show']

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=.25)

In [39]:
y_test.shape

(27632,)

In [40]:
mpipe = make_pipeline(StandardScaler(), LogisticRegressionCV(cv=3))
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('fit', LogisticRegressionCV(cv=3))
])

In [41]:
pipe.fit(X_train, y_train)

Pipeline(steps=[('scale', StandardScaler(copy=True, with_mean=True, with_std=True)), ('fit', LogisticRegressionCV(Cs=10, class_weight=None, cv=3, dual=False,
           fit_intercept=True, intercept_scaling=1.0, max_iter=100,
           multi_class='ovr', n_jobs=1, penalty='l2', random_state=None,
           refit=True, scoring=None, solver='lbfgs', tol=0.0001, verbose=0))])

In [22]:
pipe.score(X_test, y_test)

0.8004849449913144

In [42]:
gbcpipe = make_pipeline(StandardScaler(), GradientBoostingClassifier())
gbcpipe = Pipeline([
    ('scale', StandardScaler()),
    ('fit', GradientBoostingClassifier())
])

In [43]:
gbcpipe.fit(X_train, y_train)
gbcpipe.score(X_test, y_test)

0.79932686740011583

In [ ]:
X = data[valid_features].copy()
y = data['No-show'].copy()

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2)

ss = StandardScaler()
Xs_train = ss.fit_transform(X_train)
Xs_test = ss.fit_transform(X_test)


#AdaBoost
ada = AdaBoostClassifier()
ada_params = {'n_estimators': [15, 25, 50, 75],
             'learning_rate': [0.01, 0.03, 0.05, .1, 0.2]}
ada_model = GridSearchCV(ada, param_grid=ada_params, cv=5)
ada_model.fit(Xs_train, y_train)
print('best score', ada_model.best_score_)
print('best params', ada_model.best_params_)
print('test score', ada_model.score(Xs_test, y_test))
predictions = ada_model.predict(Xs_test)
acc = accuracy_score(y_test, predictions)
print('AdaBoost', acc)
#gboosting
gbc = GradientBoostingClassifier()
gbc_params = {'n_estimators': [100, 125, 150, 175], 'loss': ['deviance', 'exponential'], 'max_depth': range(1,4)}
gbc_model = GridSearchCV(gbc, param_grid=gbc_params, cv = 5)
gbc_model.fit(Xs_train, y_train)
print('best score', gbc_model.best_score_)
print('best params', gbc_model.best_params_)
print('test score', gbc_model.score(Xs_test, y_test))

predictions = gbc_model.predict(Xs_test)
acc = accuracy_score(y_test, predictions)
print('gbc', acc)
#KNeighbors
knn = KNeighborsClassifier()
knn_params = {'n_neighbors': range(2,6), 'weights': ['uniform', 'distance'], 'algorithm': ['auto', 'ball_tree']}
knn_model = GridSearchCV(knn, param_grid=knn_params, cv=5)
knn_model.fit(Xs_train, y_train)
print('best score', knn_model.best_score_)
print('best params', knn_model.best_params_)
print('test score', knn_model.score(Xs_test, y_test))
predictions = knn_model.predict(Xs_test)
acc = accuracy_score(y_test, predictions)
print('knn', acc)
#Decision Tree
dt = DecisionTreeClassifier()
dt_params = {'max_depth': range(2, 40)}
dt_model = GridSearchCV(dt, param_grid=dt_params, cv=5)
dt_model.fit(Xs_train, y_train)
print('best score', dt_model.best_score_)
print('best params', dt_model.best_params_)
print('test score', dt_model.score(Xs_test, y_test))
predictions = dt_model.predict(Xs_test)
acc = accuracy_score(y_test, predictions)
print('DecTree', acc)
##Random Forest
rf = RandomForestClassifier()
rf_params = {'max_depth': range(2, 40), 'n_estimators': range(10, 100, 10)}
rf_model = GridSearchCV(rf, param_grid=rf_params, cv=5)
rf_model.fit(Xs_train, y_train)
print('best score', rf_model.best_score_)
print('best params', rf_model.best_params_)
print('test score', rf_model.score(Xs_test, y_test))
predictions = rf_model.predict(Xs_test)
acc = accuracy_score(y_test, predictions)
print('RandomForest', acc)
##Support Vector Machines
svc = SVC()

svc_params = {'C': range(1, 5),
             'kernel': ['linear', 'rbf']}
svc_model = GridSearchCV(svc, param_grid=svc_params, cv=5)##put cv
svc_model.fit(Xs_train, y_train)
print('best score', svc_model.best_score_)
print('best params', svc_model.best_params_)
print('test score', svc_model.score(Xs_test, y_test))
predictions = svc_model.predict(Xs_test)
acc = accuracy_score(y_test, predictions)
print('SVC', acc)
##Logistic Regression
logreg = SGDClassifier()
logreg_params = {'loss': ['hinge', 'log', 'squared_hinge'],
                'penalty': ['l1', 'l2'], 'n_jobs': [-1]}
logreg_model = GridSearchCV(logreg, param_grid=logreg_params, cv=5)
logreg_model.fit(Xs_train, y_train)
print('best score', logreg_model.best_score_)
print('best params', logreg_model.best_params_)
print('test score', logreg_model.score(Xs_test, y_test))
logreg.fit(Xs_train, y_train)
predictions = logreg_model.predict(Xs_test)
acc = accuracy_score(y_test, predictions)
print('LogReg', acc)

best score 0.798673020086
best params {'learning_rate': 0.2, 'n_estimators': 75}
test score 0.796286913723
AdaBoost 0.796286913723


In [ ]:
X